In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import perm
import problem_set_3a
import time

In [3]:
perm.foo()

'hello world'

In [4]:
from problem_set_3a import *
import time
from perm import *

In [5]:
#Search for words in word_list

def bSearch(L, e, low, high):
    if high - low < 2:
        return L[low] == e or L[high] == e
    mid = low + int((high - low)/2)
    if L[mid] == e:
        return True
    if L[mid] > e:
        return bSearch(L, e, low, mid - 1)
    else:
        return bSearch(L, e, mid + 1, high)
def search(L, e):
    return bSearch(L, e, 0, len(L) -1)

In [6]:
#Testing searching if the word exists in the dictionary.
word_list = load_words()
print(search(word_list, "aksjdh"))
print(search(word_list, "love"))

Loading word list from file...
   83667 words loaded.
False
True


In [7]:
#Sort list of word by scabble score, from high to low

def merge(left, right, based_on):
    """Assumes left and right are sorted lists.
     lt defines an ordering on the elements of the lists.
     Returns a new sorted(by lt) list containing the same elements
     as (left + right) would contain."""
    result = []
    i,j = 0, 0
    while i < len(left) and j < len(right):
        if based_on(left[i], right[j]):
            result.append(left[i])
            i += 1
        else:
            result.append(right[j])
            j += 1
    while (i < len(left)):
        result.append(left[i])
        i += 1
    while (j < len(right)):
        result.append(right[j])
        j += 1
    return result
            
def sort(L, high_low_score = lambda x,y: get_word_score(x, HAND_SIZE) > get_word_score(y, HAND_SIZE)):
    """Returns a new sorted list containing the same elements as L"""
    if len(L) < 2:
        return L[:]
    else:
        middle = int(len(L)/2)
        left = sort(L[:middle], high_low_score)
        right = sort(L[middle:], high_low_score)
        #print ('About to merge', left, 'and', right)
        return merge(left, right, high_low_score)

In [8]:
# Testing sorting words by scrabble score

test_List = ['ab', 'lob', 'bar', 'babe', 'c', 'bob', 'az']
sorted_test_List = sort(test_List)
print ('Sorted list =', sorted_test_List)

Sorted list = ['babe', 'az', 'bob', 'bar', 'lob', 'ab', 'c']


In [9]:
# Problem #6A: Computer chooses a word

def comp_choose_word(hand, word_list):
    """
	Given a hand and a word_dict, find the word that gives the maximum value score, and return it.
   	This word should be calculated by considering all possible permutations of lengths 1 to HAND_SIZE.

    hand: dictionary (string -> int)
    word_list: list (string)
    """  
    perm_hand = []
    for i in range (1, (sum(hand.values()) + 1)):
        perm_hand_n = get_perms(hand, i)
        perm_hand += perm_hand_n
    legal_words = []
    for perm_word in perm_hand:
        if search(word_list, perm_word):
            legal_words += [perm_word]
        else:
            pass
    sorted_by_score = sort(legal_words)
    if len(sorted_by_score) == 0:
        return None
    else:
        return sorted_by_score[0]         

In [10]:
#Testing computer choosing a legal word with highest score. Test 1.

print(comp_choose_word({'d': 1, 'l': 1, 'e': 1, 'g': 1, 'a': 1, 'n': 1, 'j': 1}, word_list))
print(comp_choose_word({'d': 0, 'l': 1, 'e': 1, 'g': 1, 'a': 1, 'n': 0, 'j': 1}, word_list))
print(comp_choose_word({'u': 1, 'g': 1, 't': 1}, word_list))

jangled
jag
tug


In [11]:
#Testing computer choosing a legal word with highest score. Test 2.

test_hand = deal_hand(HAND_SIZE)
display_hand(test_hand)
comp_choose_word(test_hand, word_list)
print(get_word_score(comp_choose_word(test_hand, word_list), HAND_SIZE))

i u b b n r f 
35


In [12]:
# Problem #6B: Computer plays a hand

def comp_play_hand(hand, word_list):
    """
     Allows the computer to play the given hand, as follows:

     * The hand is displayed.

     * The computer chooses a word using comp_choose_words(hand, word_dict).

     * After every valid word: the score for that word is displayed, 
       the remaining letters in the hand are displayed, and the computer 
       chooses another word.

     * The sum of the word scores is displayed when the hand finishes.

     * The hand finishes when the computer has exhausted its possible choices (i.e. comp_play_hand returns None).

     hand: dictionary (string -> int)
     word_list: list (string)
    """
    total_points = 0
    while comp_choose_word(hand, word_list) is not None:
        print("Current computer's hand:", end=" ")
        display_hand(hand)
        word = comp_choose_word(hand, word_list)
        hand = update_hand(hand, word)
        total_points += get_word_score(word, HAND_SIZE)
        print('"{}" earned {} points for the computer.'.format(word, get_word_score(word, HAND_SIZE)))
        print("Total computer points: ", total_points, " points.")
        print()

In [13]:
print('a', 'b', 'c', sep="")

abc


In [15]:
#Testing computer playing a hand:

test_hand = deal_hand(HAND_SIZE)
comp_play_hand(test_hand, word_list)

Current computer's hand: u o d h j f r 
"fjord" earned 80 points for the computer.
Total computer points:  80  points.

Current computer's hand: u h 
"uh" earned 10 points for the computer.
Total computer points:  90  points.



In [19]:
# Problem #6C: Playing a game

def play_game(word_list):
    
    """Allow the user to play an arbitrary number of hands.

    1) Asks the user to input 'n' or 'r' or 'e'.
    * If the user inputs 'n', play a new (random) hand.
    * If the user inputs 'r', play the last hand again.
    * If the user inputs 'e', exit the game.
    * If the user inputs anything else, ask them again.

    2) Ask the user to input a 'u' or a 'c'.
    * If the user inputs 'u', let the user play the game as before using play_hand.
    * If the user inputs 'c', let the computer play the game using comp_play_hand (created above).
    * If the user inputs anything else, ask them again.

    3) After the computer or user has played the hand, repeat from step 1

    word_list: list (string)
    """
    hand = None
    while True:
        user_input1 = input("Enter either 'n' to play a new hand, "
                            "'r' to replay the previous hand, "
                            "or 'e' to exit game: ")
        if user_input1 == 'n':
            hand = deal_hand(HAND_SIZE)
        elif user_input1 == 'r' and hand is not None:
            pass
        elif user_input1 == 'e':
            break
        else:
            continue

        user_input2 = input("Enter either 'u' to play a hand, "
                            "or 'c' to let computer have some fun too: ")
        print()
        if user_input2 == "u":
            play_hand(hand, word_list)
        if user_input2 == 'c':
            comp_play_hand(hand, word_list)
        print

In [20]:
#Testing playing the game with computer

play_game(word_list)

Enter either 'n' to play a new hand, 'r' to replay the previous hand, or 'e' to exit game: d
Enter either 'n' to play a new hand, 'r' to replay the previous hand, or 'e' to exit game: v
Enter either 'n' to play a new hand, 'r' to replay the previous hand, or 'e' to exit game: n
Enter either 'u' to play a hand, or 'c' to let computer have some fun too: c

Current computer's hand: a a r f b b d 
"farad" earned 45 points for the computer.
Total computer points:  45  points.

Enter either 'n' to play a new hand, 'r' to replay the previous hand, or 'e' to exit game: r
Enter either 'u' to play a hand, or 'c' to let computer have some fun too: u

Current hand: a a r f b b d 
Enter word, or '.' to indicate that you are finished: bard
"bard" earned 28 points. Total: 28 points.

Current hand: a f b 
Enter word, or '.' to indicate that you are finished: fab
"fab" earned 24 points. Total: 52 points.

Total: 52 points.
Enter either 'n' to play a new hand, 'r' to replay the previous hand, or 'e' to 

In [ ]:
if __name__ == '__main__':
    word_list = load_words()
    play_game(word_list)